In [1]:
import pandas as pd
from sodapy import Socrata
import matplotlib.pyplot as plt
import yaml

In [15]:
#initial test pull, used docs: https://dev.socrata.com/foundry/data.pa.gov/dc5b-gebx
try:
    with open("config.yaml", "r") as file:
        config = yaml.safe_load(file)
except FileNotFoundError:
    print("Config file not found.")
except yaml.YAMLError as exc:
    print(f"Error in configuration file: {exc}")

client = Socrata("data.pa.gov", config['APP_TOKEN'])
results = client.get("dc5b-gebx", crash_year = '2020', limit=500000, offset=500)
test_df = pd.DataFrame.from_records(results)

#request timing out, data set is 3 million rows so probably too big -- limiting pull to 2020-Most Recent

In [5]:
test_df = pd.DataFrame.from_records(results)

In [17]:
test_df.shape

(83458, 213)